In [104]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
sys.path.insert(1, str(Path.cwd().parent))
str(Path.cwd().parent)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


'/home/ximo/Documents/GitHub/skforecast'

In [105]:
import numpy as np
import pandas as pd
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.model_selection import grid_search_forecaster
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression

In [106]:
y = pd.Series(np.random.normal(size=50))
exog_1 = pd.Series([np.nan]*25 + [2]*25, name='exog_1', dtype=float)
exog_2 = pd.Series([3]*25 + [2]*25, name='exog_2', dtype='category')
exog   = pd.concat((exog_1, exog_2), axis=1)

In [107]:
forecaster = ForecasterAutoreg(regressor=LGBMRegressor(), lags=5)

In [108]:
X_train, y_train = forecaster.create_train_X_y(
    y=y,
    exog=exog_2
)
print(X_train.dtypes)
X_train.head(3)

lag_1      float64
lag_2      float64
lag_3      float64
lag_4      float64
lag_5      float64
exog_2    category
dtype: object


,lag_1,lag_2,lag_3,lag_4,lag_5,exog_2
5,0.455417,1.531868,-0.035084,0.624972,0.354442,3
6,-0.749850,0.455417,1.531868,-0.035084,0.624972,3
7,0.502262,-0.749850,0.455417,1.531868,-0.035084,3


In [109]:
forecaster.fit(y=y, exog=exog)

/home/ximo/Documents/GitHub/skforecast/skforecast/utils/utils.py:242: UserWarning: `exog` has missing values. Most of machine learning models do not allow missing values. Fitting the forecaster may fail.
  warnings.warn(


In [110]:
exog_predict = exog.copy()
exog_predict.index = pd.RangeIndex(50, 100)
forecaster.predict(steps=3, exog=exog_predict)

/home/ximo/Documents/GitHub/skforecast/skforecast/utils/utils.py:523: UserWarning: `exog` has missing values. Most of machine learning models do not allow missing values. `predict` method may fail.
  warnings.warn(
/home/ximo/Documents/GitHub/skforecast/skforecast/utils/utils.py:242: UserWarning: `exog` has missing values. Most of machine learning models do not allow missing values. Fitting the forecaster may fail.
  warnings.warn(


50   -0.040578
51   -0.498442
52   -0.051412
Name: pred, dtype: float64

In [111]:
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector
from sklearn.preprocessing import OrdinalEncoder

categorical_cols = exog.select_dtypes(include="category").columns
ordinal_encoder = make_column_transformer(
    (
        OrdinalEncoder(dtype=int, handle_unknown="use_encoded_value", unknown_value=-1),
        make_column_selector(dtype_include="category"),
    ),
    remainder="passthrough",
    verbose_feature_names_out=False,
).set_output(transform="pandas")

exog_encoded = ordinal_encoder.fit_transform(exog)
exog_encoded[categorical_cols] = exog_encoded[categorical_cols].astype('category')

In [112]:
X_train, y_train = forecaster.create_train_X_y(
    y=y,
    exog=exog_encoded
)
X_train.head(3)

/home/ximo/Documents/GitHub/skforecast/skforecast/utils/utils.py:242: UserWarning: `exog` has missing values. Most of machine learning models do not allow missing values. Fitting the forecaster may fail.
  warnings.warn(


,lag_1,lag_2,lag_3,lag_4,lag_5,exog_2,exog_1
5,0.455417,1.531868,-0.035084,0.624972,0.354442,1,NaN
6,-0.749850,0.455417,1.531868,-0.035084,0.624972,1,NaN
7,0.502262,-0.749850,0.455417,1.531868,-0.035084,1,NaN


In [113]:
X_train.dtypes

lag_1      float64
lag_2      float64
lag_3      float64
lag_4      float64
lag_5      float64
exog_2    category
exog_1     float64
dtype: object

In [114]:
forecaster.fit(y=y, exog=exog_encoded)

/home/ximo/Documents/GitHub/skforecast/skforecast/utils/utils.py:242: UserWarning: `exog` has missing values. Most of machine learning models do not allow missing values. Fitting the forecaster may fail.
  warnings.warn(


In [115]:
exog_predict = exog_encoded.copy()
exog_predict.index = pd.RangeIndex(50, 100)
forecaster.predict(steps=3, exog=exog_predict)

/home/ximo/Documents/GitHub/skforecast/skforecast/utils/utils.py:523: UserWarning: `exog` has missing values. Most of machine learning models do not allow missing values. `predict` method may fail.
  warnings.warn(
/home/ximo/Documents/GitHub/skforecast/skforecast/utils/utils.py:242: UserWarning: `exog` has missing values. Most of machine learning models do not allow missing values. Fitting the forecaster may fail.
  warnings.warn(


50   -0.040578
51   -0.498442
52   -0.051412
Name: pred, dtype: float64

In [116]:
X=exog.copy()
model = LGBMRegressor()
model.fit(y=y.to_frame(), X=X)
model.predict(X)

array([-0.25966518, -0.25966518, -0.25966518, -0.25966518, -0.25966518,
       -0.25966518, -0.25966518, -0.25966518, -0.25966518, -0.25966518,
       -0.25966518, -0.25966518, -0.25966518, -0.25966518, -0.25966518,
       -0.25966518, -0.25966518, -0.25966518, -0.25966518, -0.25966518,
       -0.25966518, -0.25966518, -0.25966518, -0.25966518, -0.25966518,
        0.00845949,  0.00845949,  0.00845949,  0.00845949,  0.00845949,
        0.00845949,  0.00845949,  0.00845949,  0.00845949,  0.00845949,
        0.00845949,  0.00845949,  0.00845949,  0.00845949,  0.00845949,
        0.00845949,  0.00845949,  0.00845949,  0.00845949,  0.00845949,
        0.00845949,  0.00845949,  0.00845949,  0.00845949,  0.00845949])

In [117]:
model.predict(X.to_numpy())

array([-0.25966518, -0.25966518, -0.25966518, -0.25966518, -0.25966518,
       -0.25966518, -0.25966518, -0.25966518, -0.25966518, -0.25966518,
       -0.25966518, -0.25966518, -0.25966518, -0.25966518, -0.25966518,
       -0.25966518, -0.25966518, -0.25966518, -0.25966518, -0.25966518,
       -0.25966518, -0.25966518, -0.25966518, -0.25966518, -0.25966518,
        0.00845949,  0.00845949,  0.00845949,  0.00845949,  0.00845949,
        0.00845949,  0.00845949,  0.00845949,  0.00845949,  0.00845949,
        0.00845949,  0.00845949,  0.00845949,  0.00845949,  0.00845949,
        0.00845949,  0.00845949,  0.00845949,  0.00845949,  0.00845949,
        0.00845949,  0.00845949,  0.00845949,  0.00845949,  0.00845949])